In [ ]:
#import snowflake.connector, numpy, pandas
import snowflake.connector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spellchecker import SpellChecker
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import gutenberg, stopwords, webtext, reuters, brown, wordnet
nltk.download('webtext')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('omw-1.4')

#create snowflake connection
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()



In [ ]:
#query to get the data
sql = '''
select
  PVID,
  BRAND,
  CATALOG_ID,
  UPDATED_AT,
  SUBMISSION_TIME,
  SUBMISSION_STATE,
  OUTPUT_DATA :description_structured :: string as SD
from
  public.outputvariants
limit
  1
'''
cs.execute(sql)
tdf = cs.fetch_pandas_all()

#loop through each row of tdf
for index, row in tdf.iterrows():
    #save current row from SD column in a variable called sdtext
    sdtext = row['SD']
    print(sdtext)
    cleanr = re.compile('<[^>]*>') 
    sdtext = re.sub(cleanr, ' ', sdtext)
    print(sdtext)
    #create string removal exclusion list
    excludelist = "'s"
    #extract text in between two characters
    sdtext = sdtext[sdtext.find('":"'):sdtext.rfind('","')]
    print(sdtext)
    #remove more than one blankspaces when appearing together from sdtext
    sdtext = re.sub(' +', ' ', sdtext)
    print(sdtext)
    #remove \n character from sdtext
    sdtext = re.sub('\s+', ' ', sdtext)
    sdtext = sdtext.replace('\\n', '')
    print(sdtext)
    #remove all non alphabet characters from sdtext except excludelist
    sdtext = re.sub('[^a-zA-Z0-9'+excludelist+']', ' ', sdtext)
    print(sdtext)
    #remove more than one blankspaces when appearing together from sdtext
    sdtext = re.sub(' +', ' ', sdtext)
    print(sdtext)
    #remove nbsp and amp characters from sdtext
    sdtext = re.sub('nbsp|amp', ' ', sdtext)
    print(sdtext)
    
    #Conduct spell check
            
    # get the words corpus as a set for faster lookups
    word_list = set(list(nltk.corpus.webtext.words()) + list(nltk.corpus.brown.words()) + list(nltk.corpus.wordnet.words()))
    # remove extra spaces
    sdtext = " ".join(sdtext.split())
    # split text into words
    words_list = sdtext.split()
    # create empty list to store misspelled words
    misspelled = []
    # iterate through words and check spelling
    for word in words_list:
        # check if word is in the word list
        if word.lower() not in word_list:
            misspelled.append(word)

    # print misspelled words
    print(misspelled)


"""#check sdtext for spelling errors
    spell = SpellChecker()
    misspelled = spell.unknown(sdtext.split())
    print(misspelled)
    #if no spelling errors, save the result in a variable called spelloutput
    if len(misspelled) == 0:
        spelloutput = 'Spelling is correct'
    #if spelling errors, save the result in a variable called spelloutput
    else:
        spelloutput = 'Spelling is incorrect'
    #save incorrect spelling in a variable called incorrectspelling
    incorrectspelling = ', '.join(misspelled)
    print(incorrectspelling)
    #suggest correct spelling
    for word in misspelled:
        print(spell.correction(word))
        #save corrected word in variable correctouput
        correctoutput = spell.correction(word)
    print(correctoutput)

    #create an empty dataframe with columns PVID, BRAND, catalog_ID, UPDATED_AT, SUBMISSION_TIME, SUBMISSION_STATE, spelloutput, incorrectspelling
odf = pd.DataFrame(columns=['PVID', 'BRAND', 'catalog_ID', 'UPDATED_AT', 'SUBMISSION_TIME', 'SUBMISSION_STATE', 'spelloutput', 'incorrectspelling', 'correctoutput'])
#append the dataframe with the values from the current row
odf = odf.append({'PVID': row['PVID'], 'BRAND': row['BRAND'], 'catalog_ID': row['CATALOG_ID'], 'UPDATED_AT': row['UPDATED_AT'], 'SUBMISSION_TIME': row['SUBMISSION_TIME'], 'SUBMISSION_STATE': row['SUBMISSION_STATE'], 'spelloutput': spelloutput, 'incorrectspelling': incorrectspelling, 'correctoutput': correctoutput}, ignore_index=True)
print (odf)
#save the dataframe in a csv file
odf.to_csv('SDQuality.csv', index=False) """



In [ ]:
import nltk

nltk.download('all') #downloads all corpora, or you can replace 'all' with the name of specific corpora

corpora = ['webtext', 'brown', 'gutenberg', 'reuters', 'inaugural', 'state_union', 'treebank']

for c in corpora:
    if 'hypoallergenic' in nltk.corpus.__getattr__(c).words():
        print(f"The word 'hypoallergenic' is included in the {c} corpus.")
        break
else:
    print("The word 'hypoallergenic' is not included in any corpus.")
